In [2]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [3]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
openai_api_key = config['OPENAI']['OPENAI_API_KEY']

os.environ.update({'OPENAI_API_KEY': openai_api_key})

### chain 설정

In [4]:
from typing import List

from models.llm.chain import BaseChain

In [11]:
template = '''
I want you to act like a Python code generator that plots graphs. You need to write Python code based on the "plotly" library. 
I will give you the graph to draw, which can be a table or a simple description. 

Follow the three steps below.
step1. Materialize the graph to plot.
step2. Write pseudocode to draw the materialized graph.
step3. Generate the plotly library Python code based on the pseudocode.
step4. Return the path to the saved image. If no image is saved, return "No graph".

In the output, the Python code should be separated by ```.
At the end of the output, return the path to the saved image.

graph1 to draw:
===
Please plot the table below as a bar chart.
| Properties | Information |
|------------|-------------|
| **Natural Language Processing Market Size in 2022** | $15.7 billion |
| **Natural Language Processing Market Forecast 2032** | $144.9 Billion |
| **Natural Language Processing Market CAGR from 2023 to 2032** | 25.1% |
===

example1 of step1:
===
- bar chart
- Title: Natural language processing market size 2022-2032.
- With 1-year bars
- 2022: $15.7 billion
- 2032: $144.9 billion
- Annualized growth rate of 25.1
===

example1 of step2:
===
BEGIN

    IMPORT necessary libraries

    DEFINE years from 2022 to 2032
    DEFINE initial market size and growth rate

    CALCULATE market sizes for each year using growth rate

    CREATE a bar chart using years and calculated market sizes
    SET chart titles and axis titles

    SAVE the chart as an image

END
===

example1 of step3:
```python
import plotly.graph_objects as go

# Set up data
years = [f"{{year}}" for year in range(2022, 2033)]
market_size = 157  # Starting size for 2022: 157 billion dollars
growth_rate = 0.251  # Growth rate: 25.1%
market_sizes = [market_size]

# Calculate market size from 2023 to 2032
for _ in range(2022, 2032):
    market_size = market_size * (1 + growth_rate)
    market_sizes.append(market_size)

# Create a bar chart
fig = go.Figure(data=[
    go.Bar(name='NLP Market Size', x=years, y=market_sizes)
])

# Set chart layout
fig.update_layout(
    title='NLP Market Size 2022~2032',
    xaxis_title='Year',
    yaxis_title='Market Size (billion dollars)',
    yaxis_tickformat=",.0f"
)

# Save the chart as an image
fig.write_image("nlp_market_size_2022_2032.png")
```

example1 of step4: nlp_market_size_2022_2032.png

graph2 to draw:
===
{graph_to_draw}
===

example2 of step1:
'''

In [12]:
class GraphChain(BaseChain):
    def __init__(self, 
                graph_template=None, 
                input_variables:List[str]=None,
                graph_template_path='../openai_skt/models/templates/graph_prompt.json', 
                model='gpt-3.5-turbo', 
                verbose=False) -> None:
        super().__init__(template=graph_template, input_variables=input_variables, template_path=graph_template_path, model=model, verbose=verbose)

    def run(self, query:str=None):
        return super().run(graph_to_draw=query)
    
    async def arun(self,query:str=None):
        return await super().arun(graph_to_draw=query)

In [7]:
graph_chain = GraphChain(graph_template=template, input_variables=['graph_to_draw'], verbose=True)

In [8]:
# Exaple of Bar chart
graph_to_draw = '''
아래의 표를 차트로 그려주세요.
+------------+----------+
|    날짜    | 비트코인 가격 (USD) |
+------------+----------+
| 2021-01-01 | $30,000  |
+------------+----------+
| 2021-02-01 | $35,000  |
+------------+----------+
| 2021-03-01 | $50,000  |
+------------+----------+
| 2021-04-01 | $55,000  |
+------------+----------+
| 2021-05-01 | $40,000  |
+------------+----------+
| 2021-06-01 | $35,000  |
+------------+----------+
| 2021-07-01 | $32,000  |
+------------+----------+
| 2021-08-01 | $45,000  |
+------------+----------+
| 2021-09-01 | $48,000  |
+------------+----------+
'''

In [45]:
graph_to_draw = "세계지도 중 한국의 색만 붉게 그려주세요."

In [9]:
result = graph_chain.run(query=graph_to_draw)



> Entering new LLMChain chain...
Prompt after formatting:

I want you to act like a Python code generator that plots graphs. You need to write Python code based on the "plotly" library. 
I will give you the graph to draw, which can be a table or a simple description. 

Follow the three steps below.
step1. Materialize the graph to plot.
step2. Write pseudocode to draw the materialized graph.
step3. Generate the plotly library Python code based on the pseudocode.
step4. Return the path to the saved image.

In the output, the Python code should be separated by ```.
At the end of the output, return the path to the saved image.

graph1 to draw:
===
Please plot the table below as a bar chart.
| Properties | Information |
|------------|-------------|
| **Natural Language Processing Market Size in 2022** | $15.7 billion |
| **Natural Language Processing Market Forecast 2032** | $144.9 Billion |
| **Natural Language Processing Market CAGR from 2023 to 2032** | 25.1% |
===

example1 of step1:



> Finished chain.


In [23]:
print(result)

===
- line chart
- Title: Bitcoin price in 2021.
- X-axis: Dates from January to September.
- Y-axis: Bitcoin price in USD.
- Data points:
    - January 1st: $30,000
    - February 1st: $35,000
    - March 1st: $50,000
    - April 1st: $55,000
    - May 1st: $40,000
    - June 1st: $35,000
    - July 1st: $32,000
    - August 1st: $45,000
    - September 1st: $48,000
===

example2 of step2:
===
BEGIN

    IMPORT necessary libraries

    DEFINE dates from January to September
    DEFINE bitcoin prices for each date

    CREATE a line chart using dates and bitcoin prices
    SET chart titles and axis titles

    SAVE the chart as an image

END
===

example2 of step3:
```python
import plotly.graph_objects as go

# Set up data
dates = ['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01']
bitcoin_prices = [30000, 35000, 50000, 55000, 40000, 35000, 32000, 45000, 48000]

# Create a line chart
fig = go.Figure(data=go.Scat

In [13]:
start_code = result.find("```python") + len("```python")
end_code = result.rfind("```")
code = result[start_code:end_code].strip()

In [18]:
start_path = result.find('step4: ') + len('step4: ')
save_path = result[start_path:].strip() if result[start_path:].strip() != 'No graph' else None

'bitcoin_price_2021.png'

In [19]:
print(code)

import plotly.graph_objects as go

# Set up data
dates = ['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01']
bitcoin_prices = [30000, 35000, 50000, 55000, 40000, 35000, 32000, 45000, 48000]

# Create a line chart
fig = go.Figure(data=go.Scatter(x=dates, y=bitcoin_prices))

# Set chart layout
fig.update_layout(
    title='Bitcoin Price in 2021',
    xaxis_title='Date',
    yaxis_title='Bitcoin Price (USD)',
    yaxis_tickformat="$,.0f"
)

# Save the chart as an image
fig.write_image("bitcoin_price_2021.png")


In [20]:
from langchain.tools.python.tool import PythonREPLTool

In [21]:
tool = PythonREPLTool()

In [22]:
tool.run(code)

Python REPL can execute arbitrary code. Use with caution.


''